**Imports**

In [ ]:
import numpy as np
import pandas as pd
import os 

import tensorflow as tf

from sbb import Sbb

%matplotlib inline
import matplotlib.pyplot as plt

**Global variables**

In [ ]:
model_folder = '../model'

checkpoint_path = model_folder + "/sbb.ckpt"

output_size = 7
input_size = (1, 5)

In [ ]:
if not os.path.exists(model_folder):
    os.mkdir(model_folder)

In [ ]:
physical_devices = tf.config.list_physical_devices("GPU")

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

**Retrive data from system**

In [ ]:
data = pd.read_csv('../data/output.csv').dropna()
train_count = round(len(data) * 0.8)

train_data, validation_data = data.iloc[:train_count], data.iloc[train_count:]

In [ ]:
def fire(raw_data, batch_size=32):
    return_index = 0
    max_index = len(raw_data) // batch_size
    
    for i in range(1, max_index):
        t_data = raw_data[batch_size * i:batch_size * (i+1)]
        yield np.expand_dims(t_data.to_numpy()[:, :5], axis=1), np.expand_dims(t_data.to_numpy()[:, 5:], axis=1)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(raw_data=train_data),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1, output_size]))

test_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(raw_data=validation_data),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1, output_size]))

In [ ]:
sbb = Sbb(checkpoint_path, output_size, input_size)

In [ ]:
sbb.train(train_dataset, test_dataset, epochs=50, batch_size=32)

In [ ]:
sbb.evaluate(test_dataset)